In [1]:
from metric_tracker import *
import argparse
from util.time.timestamp import *
import numpy as np
import torch
import pandas as pd

In [2]:
grid_df = pd.read_csv('grid_df.csv')
regional_df = pd.read_csv('regional_df.csv')

In [8]:
def regional_leadtime_result(time_inverval=[6, 12, 18, 24, 30, 36, 42, 48], binary_classification_index=2, n_classes=4):
    # REGIONAL

    regional_result = pd.DataFrame(
        columns=['lead_time', 'nmb', 'nme', 'r', 'rmse', 'acc', 'hard_acc', 'far', 'pod', 'f1'])
    
    for start, end in zip([0]+time_inverval, time_inverval+[1000]):
        regional_df_t = regional_df[(regional_df['lead_time'] >= start) & (regional_df['lead_time'] < end)]

        if len(regional_df_t) == 0:
            continue

        # regression
        reg_m = calculate_regression_metrics(np.array(regional_df_t['prediction']), 
                                             np.array(regional_df_t['target']))

        # classification
        categorical_predictions = np.array(regional_df_t['categorical_prediction']).reshape(-1, 1).astype(int)
        categorical_targets = np.array(regional_df_t['categorical_target']).reshape(-1, 1).astype(int)
        conf_arr = calculate_confusion_matrix(categorical_predictions, categorical_targets, n_classes=n_classes)
        cat_m = calculate_classification_metrics(conf_arr, 
                                                 binary_classification_index=binary_classification_index)

        # add result
        new_data = pd.DataFrame({
            'lead_time': f"[{start}, {end})",
            'nmb': [round(reg_m['nmb']*100, 2)],
            'nme': [round(reg_m['nme']*100, 2)],
            'r': [round(reg_m['r'], 2)],
            'rmse': [round(reg_m['rmse'], 2)],

            'acc': [round(cat_m['acc']*100, 2)],
            'hard_acc': [round(cat_m['hard_acc']*100, 2)],
            'far': [round(cat_m['far']*100, 2)],
            'pod': [round(cat_m['pod']*100, 2)],
            'f1': [round(cat_m['f1']*100, 2)],
        })
        regional_result = pd.concat([regional_result, new_data], ignore_index=True)

    return regional_result

In [4]:
regional_leadtime_result()

/tmp/ipykernel_2582248/1016612665.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regional_result = pd.concat([regional_result, new_data], ignore_index=True)


,lead_time,nmb,nme,r,rmse,acc,hard_acc,far,pod,f1
0,"[6, 12)",-28.87,64.04,-0.14,43.78,28.07,27.27,22.22,47.73,59.15
1,"[12, 18)",-1.27,54.63,0.14,34.92,28.95,34.62,32.14,73.08,70.37
2,"[18, 24)",-42.45,58.49,0.33,41.24,21.05,15.38,12.50,53.85,66.67


In [9]:
def grid_leadtime_result(time_inverval=[6, 12, 18, 24, 30, 36, 42, 48], binary_classification_index=2):
    # REGIONAL

    regional_result = pd.DataFrame(
        columns=['lead_time', 'acc', 'hard_acc', 'far', 'pod', 'f1'])
    
    for start, end in zip([0]+time_inverval, time_inverval+[1000]):
        grid_df_t = grid_df[(grid_df['lead_time'] >= start) & (grid_df['lead_time'] < end)]

        if len(grid_df_t) == 0:
            continue

        # classification
        conf_arr = convert_confusion_df_to_array(grid_df_t)
        cat_m = calculate_classification_metrics(conf_arr, 
                                                 binary_classification_index=binary_classification_index)

        # add result
        new_data = pd.DataFrame({
            'lead_time': f"[{start}, {end})",
            'acc': [round(cat_m['acc']*100, 2)],
            'hard_acc': [round(cat_m['hard_acc']*100, 2)],
            'far': [round(cat_m['far']*100, 2)],
            'pod': [round(cat_m['pod']*100, 2)],
            'f1': [round(cat_m['f1']*100, 2)],
        })
        regional_result = pd.concat([regional_result, new_data], ignore_index=True)

    return regional_result

In [10]:
grid_leadtime_result()

/tmp/ipykernel_2582248/3217208908.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regional_result = pd.concat([regional_result, new_data], ignore_index=True)


,lead_time,acc,hard_acc,far,pod,f1
0,"[6, 12)",21.36,33.88,67.94,64.90,42.92
1,"[12, 18)",21.91,34.07,67.53,65.18,43.34
2,"[18, 24)",21.37,33.97,67.44,65.72,43.55
